In [19]:
import asyncio
import threading
import time
from autogpt.agents.agent import Agent, AgentConfiguration, AgentSettings
from typing import Optional
from forge.sdk.model import Task, TaskRequestBody
import uuid
from autogpt.agents.base import BaseAgent, BaseAgentConfiguration, BaseAgentSettings
from pydantic import Field
from autogpt.agent_factory.profile_generator import AgentProfileGenerator
from autogpt.agents.prompt_strategies.two_shot import (
    TwoShotAgentPromptConfiguration,
    TwoShotAgentPromptStrategy,
)
from autogpt.config import AIProfile, ConfigBuilder
from autogpt.models.command_registry import CommandRegistry
from autogpt.commands import COMMAND_CATEGORIES
from autogpt.core.resource.model_providers.hugging_chat import HuggingChatProvider
import logging
from autogpt.config import (
    AIDirectives,
    AIProfile,
    Config,
    ConfigBuilder,
    assert_config_has_openai_api_key,
)
from autogpt.core.resource.model_providers import ChatModelProvider

logger = logging.getLogger(__name__)

class TwoShotAgentSettings(BaseAgentSettings):
    config: AgentConfiguration = Field(default_factory=AgentConfiguration)
    prompt_config: TwoShotAgentPromptConfiguration = Field(
        default_factory=(
            lambda: TwoShotAgentPromptStrategy.default_configuration.copy(deep=True)
        )
    )

async def generate_agent_profile_for_task(
    task: str,
    app_config: Config
) -> TwoShotAgentSettings:
    hugging_chat_settings = HuggingChatProvider.default_settings.copy(deep=True)
    hugging_chat_settings.credentials = app_config.hugging_chat_credentials
    llm_provider = HuggingChatProvider(
        settings=hugging_chat_settings,
        logger=logger.getChild(f"HuggingChatProvider"),
    )
    agent_profile_generator = AgentProfileGenerator(
        **AgentProfileGenerator.default_configuration.dict()  # HACK
    )

    prompt = agent_profile_generator.build_prompt(task)

    output = (
        await llm_provider.create_chat_completion(
            prompt.messages,
            model_name=app_config.smart_llm,
            functions=prompt.functions,
        )
    ).response

    ai_profile, ai_directives = agent_profile_generator.parse_response_content(output)

    agent_prompt_config = Agent.default_settings.prompt_config.copy(deep=True)
    agent_prompt_config.use_functions_api = app_config.openai_functions

    return TwoShotAgentSettings(
        name=Agent.default_settings.name,
        description=Agent.default_settings.description,
        task=task,
        ai_profile=ai_profile,
        directives=directives,
        config=AgentConfiguration(
            fast_llm=app_config.fast_llm,
            smart_llm=app_config.smart_llm,
            allow_fs_access=not app_config.restrict_to_workspace,
            use_functions_api=app_config.openai_functions,
            plugins=app_config.plugins,
        ),
        prompt_config=agent_prompt_config,
        history=Agent.default_settings.history.copy(deep=True),
    )

async def create_agent(
    role: str,
    initial_prompt:str,
    boss: Optional['AgentMember'] = None,
    recruiter: Optional['AgentMember'] = None,
    create_agent: bool = False,
) -> AgentMember:
    config = ConfigBuilder.build_config_from_env()
    config.logging.plain_console_output = True

    config.continuous_mode = False
    config.continuous_limit = 20
    config.noninteractive_mode = True
    config.memory_backend = "no_memory"
    settings = await generate_agent_profile_for_task(
            task=initial_prompt,
            app_config=config,
        )
    return AgentMember(
        role=role,
        initial_prompt=initial_prompt,
        settings=settings,
        boss=boss,
        recruiter=recruiter,
        create_agent=create_agent
    )

class AgentMember(Agent):
    
    id: str
    role: str
    initial_prompt: str
    boss: Optional['AgentMember']
    recruiter: Optional['AgentMember']
    task_queue: list[Task]
    employees: list['AgentMember']
    create_agent: bool
        
    def __init__(
        self,
        role: str,
        initial_prompt: str,
        settings: TwoShotAgentSettings,
        boss: Optional['AgentMember'] = None,
        recruiter: Optional['AgentMember'] = None,
        create_agent: bool = False,
    ):
        setting = TwoShotAgentSettings(
            name="Agent",
            description=__doc__,
        )
        config = ConfigBuilder.build_config_from_env()
        config.logging.plain_console_output = True

        config.continuous_mode = False
        config.continuous_limit = 20
        config.noninteractive_mode = True
        config.memory_backend = "no_memory"

        command_registry = CommandRegistry.with_command_modules(COMMAND_CATEGORIES, config)
        
        hugging_chat_settings = HuggingChatProvider.default_settings.copy(deep=True)
        hugging_chat_settings.credentials = config.hugging_chat_credentials

        llm_provider = HuggingChatProvider(
            settings=hugging_chat_settings,
            logger=logger.getChild(f"Role-{role}_HuggingChatProvider"),
        )
        
        base_directives = AIDirectives.from_file(config.prompt_settings_file)

        

        super().__init__(settings, llm_provider, command_registry, config)
        
        self.id = str(uuid.uuid4())[:8]
        self.role = role
        self.initial_prompt = initial_prompt
        self.boss = boss
        self.recruiter = recruiter
        self.task_queue = []
        self.employees = []
        self.create_agent = create_agent
        self.get_detail_of_llm()
        
    def break_task(self, task:Task) -> tuple[dict['AgentMember', Task], list[tuple[str, dict[str, str]]]]:
        raise Exception("not implemented yet")

    def create_task(self, task_request: TaskRequestBody):
        task = self.db.create_task(
            input=task_request.input,
            additional_input=task_request.additional_input,
        )
        self.push_task(task)
        
    def run_tasks(self):
        while True:
            if self.task_queue:
                task = self.task_queue.pop()
                self.do_task(task)
            time.sleep(3)

    def do_task(self, task: Task):
        [tasks, commands] = self.break_task(task)
        for command in commands:
            self.do_command(command)
        for agent_member, sub_task in tasks.items():
            agent_member.push_task(sub_task)

    def push_task(self, task: Task):
        self.task_queue.append(task)


class AgentGroup:
    
    ceo: AgentMember
    tasks: list[Task]
        
    def __init__(
        self,
        ceo: AgentMember
    ):
        self.ceo = ceo
        self.tasks = []

    def create_task(self, task: TaskRequestBody):
        self.ceo.create_task(task)

def run_agent_tasks(agent):
    agent.run_tasks()

if __name__ == "__main__":
    ceo = await create_agent(
        role="ceo",
        initial_prompt="you are ceo of a software game company"
    )

    hr_lead = await create_agent(
        role="hr_lead",
        initial_prompt="you are hr_lead of a company You'll recruite agents when we need it",
        boss=ceo
    )

    ceo.recruiter = hr_lead

    agentGroup = AgentGroup(
        ceo=ceo
    )

    agentGroup.create_task(TaskRequestBody(input="Create best shooter game in wold", additional_input={}))

    for agent in [ceo, hr_lead]:
        agent_thread = threading.Thread(target=run_agent_tasks, args=(agent,))
        agent_thread.daemon = True
        agent_thread.start()


ERROR:root:No `type` found in response: {'message': 'Internal Error'}


ChatError: Server returns an error: Internal Error

# Create server

In [1]:
from autogpt.app.main import create_server

server = create_server()

2024-02-11 19:21:26,562 WARNING  Not loading plugin AutoGPTApiTools. No entry for 'AutoGPTApiTools' in plugins_config.yaml. Note: Zipped plugins should use the class name (AutoGPTApiTools) as the key.
2024-02-11 19:21:26,564 WARNING  Not loading plugin AutoGPTSpacePlugin. No entry for 'AutoGPTSpacePlugin' in plugins_config.yaml. Note: Zipped plugins should use the class name (AutoGPTSpacePlugin) as the key.
2024-02-11 19:21:26,566 WARNING  Not loading plugin AutoGPTBaiduSearch. No entry for 'AutoGPTBaiduSearch' in plugins_config.yaml. Note: Zipped plugins should use the class name (AutoGPTBaiduSearch) as the key.
2024-02-11 19:21:26,570 WARNING  Not loading plugin AutoGPTBingSearch. No entry for 'AutoGPTBingSearch' in plugins_config.yaml. Note: Zipped plugins should use the class name (AutoGPTBingSearch) as the key.
2024-02-11 19:21:26,572 WARNING  Not loading plugin AutoGPTBluesky. No entry for 'AutoGPTBluesky' in plugins_config.yaml. Note: Zipped plugins should use the class name (Au

# Create new task

In [2]:
from forge.sdk.model import TaskRequestBody

response = await server.create_task(TaskRequestBody(input="Write result of 2 + 2 in karam.data file"))

2024-02-09 15:14:52,829 INFO  hf-chat loaded
2024-02-09 15:14:52,830 INFO  token loaded


In [3]:
#detail of a create task response
task_id = response.task_id
print(response)


input='Write result of 2 + 2 in karam.data file' additional_input={} created_at=datetime.datetime(2024, 2, 9, 11, 44, 52, 810412) modified_at=datetime.datetime(2024, 2, 9, 11, 44, 52, 810415) task_id='5cd4102e-4e0b-4d24-ad16-d1049634d6d3' artifacts=[]


# Execute step

In [4]:
from forge.sdk.model import StepRequestBody

step_response = await server.execute_step(task_id=task_id, step_request=StepRequestBody())

In [7]:
#detail of a create step response
print(step_response)

name=None input=None additional_input={} created_at=datetime.datetime(2024, 2, 9, 11, 46, 8, 425748) modified_at=datetime.datetime(2024, 2, 9, 11, 46, 22, 731226) task_id='5cd4102e-4e0b-4d24-ad16-d1049634d6d3' step_id='1c5326c7-cf9b-4493-abbf-cf8f8c1ba9ff' status=<Status.completed: 'completed'> output='I am going to calculate the sum of 2 + 2 and save the result to the \'karam.data\' file.\n\nNext Command: execute_python_code(code="result = 2 + 2;\\nopen_file(\'karam.data\');\\nwrite_file(file_path=\'karam.data\', contents=str(result))")' additional_output={'thoughts': {'observations': "User requested writing the result of 2 + 2 to a file named 'karam.data'.", 'text': 'To accomplish this task, I first need to calculate the sum of 2 + 2, then write the result to the specified file.', 'reasoning': "Using the 'execute_python_code' command, I can efficiently compute the sum within the same command where I write to the file. This saves time compared to executing separate commands for calcul

# List steps of a task

In [14]:
response_steps = await server.list_steps(task_id=task_id)

for step in response_steps.steps:
    print(step.output)

I am going to calculate the sum of 2 + 2 and save the result to the 'karam.data' file.

Next Command: execute_python_code(code="result = 2 + 2;\nopen_file('karam.data');\nwrite_file(file_path='karam.data', contents=str(result))")


# List tasks

In [15]:
response = await server.list_tasks()
for task in response.tasks:
    print(task.task_id + ": " + task.input)


ecdc0174-9363-4a61-a0cc-1119fffb85ed: Write the word 'Washington' to a .txt file
ea002900-bf25-4c7a-95b4-4309eae5be0b: Write the word 'Washington' to a .txt file
4311c783-3fb3-49d4-8b75-87fd3fe0ba9b: Write the word 'Washington' to a .txt file
47d9147f-8374-485d-80f6-305b334d88d5: Write the word 'Washington' to a .txt file
227661d5-3bdc-406e-85f8-be360433ec70: Write the word 'Washington' to a .txt file
56fb0bf5-3df3-4157-8fb5-c89533ca67d4: Write the word 'Washington' to a .txt file
359a7751-f41d-4844-93dd-927c796080e5: Write the word 'Washington' to a .txt file
ed576fc5-f63c-432d-8608-059c90d84908: Write the word 'Washington' to a .txt file
65378dc2-4de1-4a6f-91e7-fd3c6d53d670: Write result of 3 + 2 in test.data file
691db931-3351-4856-a62d-66a5cd14571a: Write result of 3 + 2 in test.data file
